In [1]:
import numpy as np
from cudaAcceleration import CUDAAccelerator

cuda_acc = CUDAAccelerator()

# Example CUDA code
code = """
__global__ void add(int n, float *x, float *y) {
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < n) {
        y[i] = x[i] + y[i];
    }
}
"""

# Compile the CUDA code
cuda_acc.compile(code)

# Allocate memory on the device and copy data to it
n = 1024
x = np.ones(n, dtype=np.float32)
y = np.zeros(n, dtype=np.float32)

dev_x = cuda_acc.to_device(x)
dev_y = cuda_acc.to_device(y)

# Get the "add" function from the compiled module
add_func = cuda_acc.get_function("add")

# Launch the kernel on the device
block_size = 256
grid_size = (n + block_size - 1) // block_size
add_func(np.int32(n), dev_x, dev_y, block=(block_size,1,1), grid=(grid_size,1))

# Copy the result back to the host
result = cuda_acc.from_device(dev_y, shape=y.shape)

print(result)


ModuleNotFoundError: No module named 'pycuda'